In [1]:
import ipaddress
import re
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import socket
import os
import requests
from googlesearch import search
from datetime import date, datetime
import csv
import time
from dateutil.parser import parse as date_parse
from urllib.parse import urlparse

In [2]:
def UsingIP(url):
    try:
        ipaddress.ip_address(url)
        return 1
    except:
        return 0

def symbol(url):
    if re.findall("@",url):
        return -1
    else:
        return 1

def redirect(response):
    try:
        if len(response.history) > 0:
            return -1
        else:
            return 1
    except requests.exceptions.RequestException as e:
        return 0

In [3]:
def http(url):
    try:
        https = url.scheme
        if "https" in https:
            return 1
        return -1
    except:
            return 1

def AnchorURL(response):
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
            i,unsafe = 0,0
            for a in soup.find_all('a', href=True):
                if "#" in a['href'] or "javascript" in a['href'].lower() or "mailto" in a['href'].lower() or not (url in a['href'] or self.domain in a['href']):
                    unsafe = unsafe + 1
                i = i + 1

            try:
                percentage = unsafe / float(i) * 100
                if percentage < 31.0:
                    return 1
                elif ((percentage >= 31.0) and (percentage < 67.0)):
                    return 0
                else:
                    return -1
            except:
                return -1

        except:
            return -1

In [4]:
def WebsiteTraffic(url):
        try:
            rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
            if (int(rank) < 100000):
                return 1
            return 0
        except :
            return -1


def longUrl(url):
    if len(url) < 54:
        return 1
    if len(url) >= 54 and len(url) <= 75:
        return 0
    return -1

In [5]:
def shortUrl(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net', url)
    if match:
        return 1
    return 0

###Specified Headers and Extra Features

In [6]:
specified_headers = [
    "Content-Security-Policy",
    "Strict-Transport-Security",
    "X-XSS-Protection",
    "X-Frame-Options",
    "X-Content-Type-Options",
    "Referrer-Policy",
    "Expect-CT",
    "Cross-Origin-Opener-Policy",
    "Public-Key-Pins",
    "X-AspNetMvc-Version",
    "Federated-Learning-Of-Cohorts",
    "X-DNS-Prefetch-Control",
    "Cross-Origin-Resource-Policy",
    "Cross-Origin-Embedder-Policy",
    "Content-Type",
    "Access-Control-Allow-Origin"
]

extra_features = [
    "Using IP",     ##Extra Features
    "Symbol",
    "Redirecting",
    "HTTPS",
    "Anchor URL",
    "Website Traffic",
    "Long URL",
    "Short URL"
]

In [7]:
with open('/content/drive/MyDrive/Dataset/Header_Data/Phishing_verified.csv', mode='r') as legit_sites:
    site_data = pd.read_csv('/content/drive/MyDrive/Dataset/Header_Data/Phishing_verified.csv', skiprows=range(1, 40000), nrows = 1500)

    # Create a CSV file to store the results
    with open("phishing_test.csv", mode="w", newline='') as output_csv_file:
        csv_writer = csv.writer(output_csv_file)
        csv_writer.writerow(["url"] + specified_headers + extra_features + ["label"])  # Write header row

        # Reset the file pointer to the beginning of the file
        #phishing_sites.seek(0)

        # Initialize count variable
        count = 0

        # Iterate over each row in the input CSV file
        for domain in site_data['url']:
            count += 1  # Increment count

            url = domain.strip()  # Extract URL from the "URLs" column

            try:
                print(f"Processing entry {count}")
                print(f"Fetching URL: {url}")
                response = requests.get(url, timeout=10)
                print(f"Response received from URL: {url}")

                # Initialize a list to hold the results for each header
                header_results = [url]

                # Check each header returned in the response
                for header in specified_headers:
                    value = response.headers.get(header, '')
                    if value:
                        header_results.append(1)
                    else:
                        header_results.append(0)

                header_results.append(UsingIP(url))
                header_results.append(symbol(url))
                header_results.append(redirect(response))
                header_results.append(http(urlparse(url)))
                header_results.append(AnchorURL(response))
               # header_results.append(WebsiteTraffic(url))
                header_results.append(0)
                header_results.append(longUrl(url))
                header_results.append(shortUrl(url))
                header_results.append(-1)

                # Write the results to the CSV file
                csv_writer.writerow(header_results)

                print("-" * 50)

            except requests.exceptions.Timeout:
                print(f"Request timed out for URL: {url}")
            except requests.exceptions.TooManyRedirects:
                print(f"Too many redirects for URL: {url}")
            except requests.exceptions.RequestException as e:
                print(f"Request exception occurred for {url}: {e}")
            except requests.exceptions.HTTPError as e:
                print(f"HTTP error occurred for {url}: {e}")
            except UnicodeDecodeError as e:
                print(f"UnicodeDecode error occurred for {url}: {e}")
            except Exception as e:
                print(f"An exception occurred for {url}: {e}")


Streaming output truncated to the last 5000 lines.
Response received from URL: https://etrgjnrethdkjnjekfv.web.app/
--------------------------------------------------
Processing entry 242
Fetching URL: https://etrgjnrethdkjnjekfv.firebaseapp.com/
Response received from URL: https://etrgjnrethdkjnjekfv.firebaseapp.com/
--------------------------------------------------
Processing entry 243
Fetching URL: https://fautvoirpourcrois-9d6b1.web.app/
Response received from URL: https://fautvoirpourcrois-9d6b1.web.app/
--------------------------------------------------
Processing entry 244
Fetching URL: https://fautvoirpourcrois-9d6b1.firebaseapp.com/
Response received from URL: https://fautvoirpourcrois-9d6b1.firebaseapp.com/
--------------------------------------------------
Processing entry 245
Fetching URL: https://hty-fv.web.app/
Response received from URL: https://hty-fv.web.app/
--------------------------------------------------
Processing entry 246
Fetching URL: https://hty-fv.firebasea

In [8]:
from google.colab import files
files.download('/content/unlegit_results_9.csv')

FileNotFoundError: Cannot find file: /content/unlegit_results_9.csv

In [ ]:
site_data = pd.read_csv('/content/drive/MyDrive/Dataset/Header_Data/majestic_million.csv', skiprows=range(1, 100000), nrows=5000)
site_data.head()